<h1><font color="#113D68" size=6>Deep Learning con Python y Keras</font></h1>

<h1><font color="#113D68" size=5>Parte 3. Multilayer Perceptron</font></h1>

<h1><font color="#113D68" size=4>Proyecto de regresión</font></h1>
<h1><font color="#113D68" size=4>Regression project</font></h1>

<br><br>
<div style="text-align: right">
<font color="#113D68" size=3>Ayelén Stefanía Andrade</font><br>

</div>

---

<a id="indice"></a>
<h2><font color="#004D7F" size=5>Índice</font></h2>

* [0. Contexto](#section0)
* [1. Topología de linea base](#section1)
* [2. Optimizar el rendimiento con procesamiento de datos](#section2)
* [3. Ajuste de la toplogía](#section3)
    * [3.1. Evaluar una topología más profunda](#section3.1)
    * [3.2. Evaluar una topología más grande](#section3.2)

---
<a id="section0"></a>
# <font color="#004D7F" size=6> 0. Contexto</font>

En este tutorial del proyecto, trabajaremos cómo desarrollar y evaluar modelos de redes neuronales para un problema de regresión:
* Cómo crear un modelo de red neuronal para un problema de regresión.
* Cómo utilizar scikit-learn con Keras para evaluar modelos mediante validación cruzada.
* Cómo realizar la preparación de datos para mejorar la habilidad.
* Cómo Optimizar la topología de red de modelos.

In [22]:
import tensorflow as tf
# Eliminar warning
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section1"></a>
# <font color="#004D7F" size=6>1. Topología de linea base</font>

En esta sección crearemos un modelo de red neuronal de referencia para el problema de regresión. Comencemos importando todas las funciones y objetos que necesitaremos para este tutorial.

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i>
Más información sobre el dataset [Boston House Price](http://lib.stat.cmu.edu/datasets/boston)

In [23]:
# Binary Classification with Sonar Dataset: Baseline
# Binary Classification with Sonar Dataset: Baseline
import pandas as pd 
from keras.models import Sequential
from keras.layers import Dense
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold


# load dataset
dataframe = pd.read_csv('Datasets/housing.csv',delim_whitespace=True, header = None)
dataset = dataframe.values
# split into input (X) and output (Y) variables
x = dataset[:, 0:13]
y = dataset[:,13]

Comencemos por definir la función que crea nuestro modelo de línea de base. 
1. Riene una sola capa oculta completamente conectada con el mismo número de neuronas que los atributos de entrada (13). 
2. La red utiliza la función de activación ReLu para la capa oculta. 
3. Utiliza el algoritmo de optimización ADAM y se optimiza una función de pérdida de error cuadrático medio. 
4. El objeto Wrapper para regresión se llama `KerasRegressor`. 
5. Evaluamos este modelo de línea de base con 10-fold.

In [25]:
# define base model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(13, input_dim = 13, activation = 'relu'))
    model.add(Dense(1))
    # Compile model
    model.compile(loss = 'mean_squared_error', optimizer = 'adam', metrics = ['accuracy'])
    return model
# evaluate model
estimator = KerasRegressor(model=baseline_model, epochs=100, batch_size=5, verbose=0)
kfold = KFold(n_splits=10)
results = cross_val_score(estimator, x, y, cv=kfold)
print("Linea base: %2.f%% (%2.f%%)" % (results.mean()*100, results.std()*100))

Linea base: -97% (242%)


---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section2"></a>
# <font color="#004D7F" size=6>2. Optimizar el rendimiento con procesamiento de datos</font>

Podemos utilizar el marco de trabajo [`Pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) para realizar la **estandarización** durante el proceso de evaluación del modelo, dentro de cada fold. 

In [26]:
# Libraries
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


# evaluate model with standardized dataset
estimators = []
estimators.append(('standarize', StandardScaler()))
estimators.append(('NN',  KerasRegressor(model=baseline_model, epochs=100, batch_size=5, verbose=0)))
kfold = KFold(n_splits=10)
results = cross_val_score(estimator, x, y, cv=kfold)
print("Estandarizado: %2.f (%2.f) MSE" % (results.mean(), results.std()))

Estandarizado: -0 ( 1) MSE


<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i>
**Trabajo a realizar:** Una extensión adicional de esta sección sería aplicar de manera similar un cambio de escala a la variable de salida, como normalizarla al rango de 0 a 1 y usar una función de activación Sigmoide o similar en la capa de salida para reducir las predicciones de salida al mismo rango.

---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section3"></a>
# <font color="#004D7F" size=6>3. Ajuste de la toplogía</font>

En esta sección evaluaremos dos topologías (más profunda y amplia) para mejorar aún más el rendimiento del modelo.

<a id="section3.1"></a>
# <font color="#004D7F" size=5>3.1. Evaluar una topología más profunda</font>

Una forma de mejorar el rendimiento de una red neuronal es agregar más capas para hacerla más profunda. En este caso con aproximadamente la mitad del número de neuronas. Nuestra topología de red ahora se ve así:
```
    13 entradas -> [13 -> 6] -> 1 salida
```
Podemos evaluar esta topología de red de la misma manera que anteriormente, al mismo tiempo que usamos la estandarización que demostró un mejor rendimiento.

In [27]:
# define the model
def larger_model():
    # create model
    model = Sequential()
    model.add(Dense(13, input_dim = 13, activation = 'relu'))
    model.add(Dense(6, activation = 'relu'))
    model.add(Dense(1))
    # Compile model
    model.compile(loss = 'mean_squared_error', optimizer = 'adam', metrics = ['accuracy'])
    return model
# evaluate model
estimators = []
estimators.append(('standarize', StandardScaler()))
estimators.append(('NN',  KerasRegressor(model=larger_model, epochs=100, batch_size=5, verbose=0)))
kfold = KFold(n_splits=10)
results = cross_val_score(estimator, x, y, cv=kfold)
print("Larger: %2.f (%2.f) MSE" % (results.mean()*100, results.std()*100))

Larger: -56 (97) MSE


<a id="section3.2"></a>
# <font color="#004D7F" size=5>3.2. Evaluar una topología más grande</font>

Otro enfoque para aumentar la capacidad de representación del modelo es crear una red más amplia. Aquí, hemos aumentado el número de neuronas en la capa oculta en de 13 a 20. La topología de nuestra red más amplia se puede resumir de la siguiente manera:

```
    13 entradas -> [20] -> 1 salida
```

In [28]:
# define the model
def wider_model():
    # create model
    model = Sequential()
    model.add(Dense(20, input_dim = 13, activation = 'relu'))
    model.add(Dense(1))
    # Compile model
    model.compile(loss = 'mean_squared_error', optimizer = 'adam', metrics = ['accuracy'])
    return model
# evaluate model
estimators = []
estimators.append(('standarize', StandardScaler()))
estimators.append(('NN',  KerasRegressor(model=wider_model, epochs=100, batch_size=5, verbose=0)))
kfold = KFold(n_splits=10)
results = cross_val_score(estimator, x, y, cv=kfold)
print("Wider: %2.f (%2.f) MSE" % (results.mean()*100, results.std()*100))

Wider: -18 (57) MSE


La construcción del modelo ve una caída adicional en el error a aproximadamente 22 mil dólares cuadrados. Este no es un mal resultado para este problema.

<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<div style="text-align: right"> <font size=6><i class="fa fa-coffee" aria-hidden="true" style="color:#004D7F"></i> </font></div>